# Create a Grafana Data Source and Dashboard

Follow the steps in this this tutorial to create a "stocks" Grafana dashboard using Iguazio data source <br>
Note that you need to create a Grafana service before running the notebook <br>
Creating a grafana service can be done from the "services" screen in the Platform

In [ ]:
!pip install git+https://github.com/v3io/grafwiz.git

In [ ]:
from grafwiz import *
import os

In [ ]:
# Grafana internal cluster address (will be http://grafana)
grafana_url = 'http://grafana-roy-external-users.default-tenant.app.dev83.lab.iguazeng.com'
# External URL of the demo's stream-viewer Nuclio serverless function

streamview_url = 'https://webapi.default-tenant.app.dev83.lab.iguazeng.com'

In [ ]:
# Create a new Grafana dashboard
dash = Dashboard("stocks", start='now-1d', dataSource='iguazio')

# Add a symbol combo box (template) with data from the stocks table
dash.template(name="SYMBOL", label="Symbol", query="fields=symbol;table=stocks;backend=kv;container=bigdata")

# Create a table and log viewer in one row
tbl = Table('tbl1',span=8).source(table='stocks',fields=['symbol','volume', 'price','last_updated'],container='bigdata')
log = Ajax(title='Log',url=streamview_url)
dash.row([tbl,log])

# Create 3 charts on the second row
dash.row([Graph(metric).series(table="stock_metrics", fields=[metric], filter='symbol=="$SYMBOL"',container='bigdata') for metric in ['price','volume','sentiment']])

# Deploy to Grafana
dash.deploy(grafana_url)